## pre-trained word embedding
- word embedding의 종류로는 여러 가지가 존재합니다.
  1. Embedding Layer를 랜덤 초기화하여 처음부터 학습하는 방식
  2. Word2Vec과 같은 pre-trained word embedding vector를 가져와 사용하는 방식
    - 해당 방식은 동음이의어를 구분할 수 없다는 단점이 존재합니다.
    - 하지만, 이것을 pre-trained language model로 해결할 수 있게 됩니다.


## BERT Tokenizer
- Bert는 Tokenizer로 서브워드 토크나이저를 사용합니다.
  - 서브워드 토크나이저
    - 단어를 찾지 못하더라도 토크나이즈를 종료하는 것이 아닌, 서브워드를 찾는 것입니다.
      - ex) kilwon을 찾지 못하더라도 ki,l,won을 찾았다면, ki,##l,##won으로 tokenize합니다.

In [ ]:
import pandas as pd
from transformers import BertTokenizer # Bert의 Tokenizer를 사용하기 위해 import 합니다.

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
result = tokenizer.tokenize('Hi I\'m kilwon. It\'s a Happy Day.')
print(result)

['hi', 'i', "'", 'm', 'ki', '##l', '##won', '.', 'it', "'", 's', 'a', 'happy', 'day', '.']


In [ ]:
# 실제 단어가 단어 집합에 존재하는지 확인해봅니다.
tokenizer.vocab["happy"]

3407

In [ ]:
# BERT의 vocab을 확인해보기 위해 txt파일로 저장합니다.
with open("bert_vocab.txt","w") as f :
  for word in tokenizer.vocab.keys():
    f.write(word+"\n")

In [ ]:
# txt파일을 확인합니다.
df = pd.read_fwf("bert_vocab.txt",header=None)
df.tail(20)

,0
30502,##面
30503,##風
30504,##食
30505,##香
30506,##馬
30507,##高
30508,##龍
30509,##龸
30510,##ﬁ
30511,##ﬂ


## Position Embedding
- BERT는 Transformer, GPT-1과 다르게 Position Embedding으로 sinusoidal function을 사용하는 것이 아니라, learnable position embedding을 사용합니다.
- postion embedding IDEA
  - word embedding을 거친 결과에 position embedding layer를 거치도록 처리합니다.
    - 따라서 각 단어마다 position embedding layer를 거쳐야 하기에 max_seq_len만큼 position embedding vector를 만들어야합니다.
    - BERT는 max_len을 512로 제한하므로 512개의 position embedding vector가 필요합니다.

## BERT Pre-train
- There are two pre-train methods for BERT
  1. masked language model
  2. Next sentence prediction (NSP)

### masked language model
- input sentence의 15%를 randomly masking하고, 이것만 학습에 사용합니다.
  - 15%는 3가지로 다시 구분하여 마스킹됩니다.
    - 80% : mark [mask]
    - 10% : change word
    - 10% : 미 변경 후 예측
      - 이렇게 3가지로 나누는 이유는 파인튜닝 과정에서는 [mask]토큰이 나타나지 않으므로 학습과 fine-tune 과정에서 불일치가 발생하기 때문입니다.
  - 기존의 my name is kilwon 이라는 문장이 있다 가정합니다.
    - 만약 kilwon을 mask 하여 my name is [mask] 가 되었다고 할 때, BERT의 output layer는 모든 단어에 대해 사용되는 것이 아니라, mask된 단어에 대해서만 사용됩니다.
      - 따라서 kilwon에 해당하는 output layer만 예측과 학습에 사용됩니다.
    - 만약, 3가지가 모두 적용되어 my food is [mask]가 된다고 가정합니다. 여기서 my는 미 변경 후, 예측입니다.
      - 이때도 my, food, [mask]에 해당하는 output layer만 예측과 학습에 사용이 됩니다.
  
### Next Sentence Prediction NSP
- BERT는 2가지 문장을 받은 후, 이 두 문장이 이어지는지 혹은 이어지지않는지를 맞추도록 훈련합니다.
  - 이때, 앞 부분에 [CLS] 토큰을 그리고, 두 문장을 분리하는 [SEP] 토큰을 첫 문장 뒤, 두번째 문장 뒤에 각각 붙입니다.
    - [CLS]토큰은 BERT가 분류 문제를 풀기 위해 추가된 특별한 토큰으로 [CLS]에 해당하는 Output layer는 이어지는 두 문장이 이어지는지 아닌지를 예측하도록합니다.

### relation of two methods
- masked language model과 NSP는 따로 학습하는 것이 아닌 같이 합쳐서 학습합니다.
- MLM외에도 NSP를 하는 이유로는 BERT의 목적 task 중 QA,NLI와 같이 두 문장의 관계를 알아야 하는 것들이 있기 때문입니다.


## Segment Embedding
- BERT는 QA,NLI 같은 두 문장의 입력이 필요한 Task를 풀기도 합니다.
  - 문장 구분을 위해서 BERT는 Segment embedding을 introduce합니다.
    - segment embedding은 구분이 목적이므로 2개의 vector를 가집니다.
      - 여기서 **문장 구분**은 .으로 구분하는 것이 아닌 2개의 문서를 구분하는 것입니다.
      - 또한, 두 개의 문장을 받지 않는 리뷰 분류 같은 task는 한 개의 문서를 다루므로 embedding으로 sentence 0 embedding 1개만 전체에 적용합니다.
- BERT는 총 3개의 embedding을 사용합니다.
  1. word embedding (단어집합의 크기로 embedding vector : 30522)
  2. position embedding (seq_len로 512)
  3. segment embedding (두 문장 구분으로 2)

## BERT Finetuning
